# Kepler Framework, Emulate v16

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [7]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [8]:
tpath_data17 = 'output/emulation/data17_table_v16.h5'
tpath_data18 = 'output/emulation/data18_table_v16.h5'
tpath_mc16   = 'output/emulation/mc16_table_v16.h5'

## Setup Chains:

In [9]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v16_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v16_ivarloose" , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v16_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v16_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v16"          , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v16_loose' ),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v16_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v16_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v16_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v16_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [10]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [11]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:36<00:00, 24.17s/it]


In [12]:
data17_df.shape

(43311283, 112)

### Validate:

In [13]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v16.root')

2022-03-24 21:22:28,299 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v16.root


In [14]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.48s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5288  | 29973692/32047563 |
|   L2   | 93.2860  | 29895898/32047563 |
| EFCalo | 91.9752  | 29475810/32047563 |
|  HLT   | 90.0326  | 28853247/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.37s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2459  | 21455642/22765602 |
|   L2   | 94.1297  | 21429197/22765602 |
| EFCalo | 93.2048  | 21218638/22765602 |
|  HLT   | 87.0196  | 19810534/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.1950  | 1329419/1367786 |
|   L2   | 97.1042  | 1328178/1367786 |
| EFCalo | 95.0136  | 1299583/1367786 |
|  HLT   | 91.7399  | 1254805/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.43it/s]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.9386  | 59293/60541  |
|   L2   | 97.8692  | 59251/60541  |
| EFCalo | 96.5494  | 58452/60541  |
|  HLT   | 94.5954  | 57269/60541  |
+--------+----------+--------------+


In [15]:
eff_data17_Zee.save()

In [16]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v16.root')

2022-03-24 21:26:41,871 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v16.root


In [17]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v16'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.35s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  1.3255  | 137185/10349992  |
|   L2   |  1.2673  | 131169/10349992  |
| EFCalo |  1.1058  | 114455/10349992  |
|  HLT   |  0.0974  |  10078/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.91s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.6511  |  60252/9254300  |
|   L2   |  0.6046  |  55949/9254300  |
| EFCalo |  0.4781  |  44244/9254300  |
|  HLT   |  0.0014  |   127/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.31s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.6738  |  52908/7852181  |
|   L2   |  0.6178  |  48514/7852181  |
| EFCalo |  0.4586  |  36008/7852181  |
|  HLT   |  0.0016  |   123/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  1.0447  |  58859/5634041  |
|   L2   |  0.9649  |  54363/5634041  |
| EFCalo |  0.7459  |  42023/5634041  |
|  HLT   |  0.0222  |   1251/5634041  |
+--------+----------+-----------------+


In [18]:
eff_data17_JF17.save()

In [19]:
del data17_df

## Emulate 2018:

In [20]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [21]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:31<00:00, 22.94s/it]


In [22]:
data18_df.shape

(41873973, 112)

In [23]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v16.root')

2022-03-24 21:30:50,188 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v16.root


In [24]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:51<00:00, 22.32s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5741  | 30524329/32620495 |
|   L2   | 93.3368  | 30446914/32620495 |
| EFCalo | 92.0034  | 30011967/32620495 |
|  HLT   | 90.5594  | 29540934/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.30s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2171  | 21965612/23313826 |
|   L2   | 94.1208  | 21943148/23313826 |
| EFCalo | 93.3208  | 21756639/23313826 |
|  HLT   | 87.5961  | 20422008/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.2255  | 1343340/1381675 |
|   L2   | 97.1355  | 1342097/1381675 |
| EFCalo | 95.0463  | 1313231/1381675 |
|  HLT   | 92.0656  | 1272047/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.39it/s]


HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 97.9665  | 59161/60389  |
|   L2   | 97.8705  | 59103/60389  |
| EFCalo | 96.6898  | 58390/60389  |
|  HLT   | 94.8766  | 57295/60389  |
+--------+----------+--------------+


In [25]:
eff_data18_Zee.save()

In [26]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v16.root')

2022-03-24 21:35:02,578 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v16.root


In [27]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v16'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.52s/it]


HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  1.6912  |  143192/8467098 |
|   L2   |  1.6327  |  138244/8467098 |
| EFCalo |  1.4734  |  124757/8467098 |
|  HLT   |  0.4906  |  41537/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.19s/it]


HLT_e28_lhtight_nod0_ringer_v16_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.9599  |  73226/7628804  |
|   L2   |  0.9159  |  69872/7628804  |
| EFCalo |  0.7952  |  60668/7628804  |
|  HLT   |  0.0405  |   3086/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]


HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.9593  |  62949/6561985  |
|   L2   |  0.9061  |  59457/6561985  |
| EFCalo |  0.7556  |  49580/6561985  |
|  HLT   |  0.0482  |   3164/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.04s/it]

HLT_e140_lhloose_nod0_ringer_v16
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  1.4374  |  68658/4776518  |
|   L2   |  1.3564  |  64790/4776518  |
| EFCalo |  1.1460  |  54740/4776518  |
|  HLT   |  0.2763  |  13199/4776518  |
+--------+----------+-----------------+


In [28]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [29]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

FileNotFoundError: File ../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5 does not exist

In [ ]:
emulate(mc16_df)

In [ ]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v16.root')

In [ ]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v16_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v16_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v16_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v16'          , pidname='el_lhloose' , reset=True)

In [ ]:
eff_mc16.save()